<a href="https://colab.research.google.com/github/sunilgug/Quant/blob/master/develop_strategy_using_visualization_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# ***Develop a new strategy using visualization***

---

#### ***1. Installation of libraries***
#### ***2. Import the libraries and download the data***

#### ***3. Technical indicators/variables***
  ##### ***a. variables we want to use for developing the strategy***
  ##### ***b. variables we need to use for backtesting of the strategy***
#### ***4. Download the dataframe in the excel file***
#### ***5. Use Tableau to derive parameters of the indicators for your strategy***
#### ***6. Analysis of performance of the strategy***










#### ***installation of library***
##### yfinance *for Data Import*
##### ta *for technical indicators*


In [ ]:
!pip install yfinance
#!pip install ta

#### ***Import the libraries and download the data***
##### Import the libraries
##### Download the data


In [ ]:
  import pandas as pd
  import yfinance as yf
  import numpy as np
  import datetime
  import warnings
  
  warnings.filterwarnings(action='ignore')


In [ ]:
def fetch_historical_data(script,period,interval):
    df=yf.download(script,period=period,interval=interval)
    return df

#### ***variables we want to use for developing the strategy***
##### vwap=sum(price*volume)/sum(volume)
##### sma_short,sma_long
##### sma_ratio=sma_short/sma_long
##### vol_pct_change
##### vwap_price_diff=(close-vwap)*100/close
##### RSI_14




In [ ]:
def RSI(df, base="Close", period=14):
    delta = df[base].diff()
    up, down = delta.copy(), delta.copy()

    up[up < 0] = 0
    down[down > 0] = 0
    
    rUp = up.ewm(com=period - 1,  adjust=False).mean()
    rDown = down.ewm(com=period - 1, adjust=False).mean().abs()

    df['RSI_' + str(period)] = 100 - 100 / (1 + rUp / rDown)
    df['RSI_' + str(period)].fillna(0, inplace=True)

    return df

In [ ]:
def load_indicators(df,long,short,lookback):
  df['sma_short']=df['Close'].rolling(short).mean()
  df['sma_long']=df['Close'].rolling(long).mean()
  df['sma_ratio']=df['sma_short']/df['sma_long']

  df['vol_pct_change_0']=df['Volume'].pct_change()*100

  df['volume_vwap']=((df['Close']*df['Volume']).rolling(lookback).sum())/df['Volume'].rolling(lookback).sum()
  df['vwap_price_diff']=np.round((df['Close']-df['volume_vwap'])*100/df['Close'],2)
  df['weekday']=pd.to_datetime(df.index)
  df['weekday']=df['weekday'].dt.dayofweek

  df=RSI(df,period=lookback)

  #df.dropna(inplace=True)

  return df

#### ***variables we need to use for backtesting of the strategy***
##### pct_change_0=pct_change of the day when position initaited
##### pct_change_1,pct_change_2 are the next 1,2 days of pct change respectively



In [ ]:
def update_pct_change(df):
  df['pct_change']=df['Close'].pct_change()*100
  df['pct_change_0']=df['pct_change']
  df['pct_change_1']=df['pct_change'].shift(-1)
  #df.dropna(inplace=True)
  return df

#### ***Download the dataframe in the excel file***
##### Download the dataframe in the excel file 
##### Use google.colab.files to download file in the local drive

In [ ]:
#fetch historical data
script,period,interval='^NSEI','59d','15m'
df=fetch_historical_data(script,period,interval)

[*********************100%***********************]  1 of 1 completed


In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2020-08-27 09:15:00+05:30,11609.299805,11616.500000,11593.950195,11608.349609,11608.349609,0
2020-08-27 09:30:00+05:30,11608.599609,11612.000000,11588.599609,11592.599609,11592.599609,0
2020-08-27 09:45:00+05:30,11591.950195,11601.400391,11587.150391,11592.299805,11592.299805,0
2020-08-27 10:00:00+05:30,11592.099609,11598.950195,11586.250000,11596.950195,11596.950195,0
2020-08-27 10:15:00+05:30,11596.700195,11603.750000,11589.349609,11594.799805,11594.799805,0
...,...,...,...,...,...,...
2020-11-19 14:15:00+05:30,12861.099609,12862.049805,12761.599609,12761.750000,12761.750000,0
2020-11-19 14:30:00+05:30,12761.450195,12838.650391,12746.450195,12830.099609,12830.099609,0
2020-11-19 14:45:00+05:30,12830.650391,12840.500000,12799.250000,12816.799805,12816.799805,0


In [ ]:
#load indicators
lookback,short,long=14,5,12
df=load_indicators(df,long,short,lookback)
df

,Open,High,Low,Close,Adj Close,Volume,sma_short,sma_long,sma_ratio,vol_pct_change_0,volume_vwap,vwap_price_diff,weekday,RSI_14
Datetime,,,,,,,,,,,,,,
2020-08-27 09:15:00+05:30,23511.449219,23643.300781,23497.099609,23637.900391,23637.900391,0,NaN,NaN,NaN,NaN,NaN,NaN,3,0.000000
2020-08-27 09:30:00+05:30,23639.349609,23649.900391,23571.500000,23603.150391,23603.150391,0,NaN,NaN,NaN,NaN,NaN,NaN,3,0.000000
2020-08-27 09:45:00+05:30,23602.199219,23628.800781,23582.599609,23587.900391,23587.900391,0,NaN,NaN,NaN,NaN,NaN,NaN,3,0.000000
2020-08-27 10:00:00+05:30,23585.949219,23618.199219,23569.550781,23593.449219,23593.449219,0,NaN,NaN,NaN,NaN,NaN,NaN,3,1.263418
2020-08-27 10:15:00+05:30,23592.599609,23625.050781,23581.300781,23605.400391,23605.400391,0,23605.560156,NaN,NaN,NaN,NaN,NaN,3,4.074511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-19 14:15:00+05:30,29109.449219,29111.349609,28861.449219,28863.849609,28863.849609,0,29161.059766,29343.845866,0.993771,NaN,NaN,NaN,3,23.448736
2020-11-19 14:30:00+05:30,28865.150391,29120.199219,28831.449219,29102.000000,29102.000000,0,29116.219922,29313.308431,0.993276,NaN,NaN,NaN,3,38.622103
2020-11-19 14:45:00+05:30,29102.199219,29150.849609,28984.550781,29036.050781,29036.050781,0,29062.600000,29278.575195,0.992623,NaN,NaN,NaN,3,36.466510


In [ ]:
#update pct change of curr and next day
df=update_pct_change(df)
df

,Open,High,Low,Close,Adj Close,Volume,sma_short,sma_long,sma_ratio,vol_pct_change_0,volume_vwap,vwap_price_diff,weekday,RSI_14,pct_change,pct_change_0,pct_change_1
Datetime,,,,,,,,,,,,,,,,,
2020-08-27 09:15:00+05:30,23511.449219,23643.300781,23497.099609,23637.900391,23637.900391,0,NaN,NaN,NaN,NaN,NaN,NaN,3,0.000000,NaN,NaN,-0.147010
2020-08-27 09:30:00+05:30,23639.349609,23649.900391,23571.500000,23603.150391,23603.150391,0,NaN,NaN,NaN,NaN,NaN,NaN,3,0.000000,-0.147010,-0.147010,-0.064610
2020-08-27 09:45:00+05:30,23602.199219,23628.800781,23582.599609,23587.900391,23587.900391,0,NaN,NaN,NaN,NaN,NaN,NaN,3,0.000000,-0.064610,-0.064610,0.023524
2020-08-27 10:00:00+05:30,23585.949219,23618.199219,23569.550781,23593.449219,23593.449219,0,NaN,NaN,NaN,NaN,NaN,NaN,3,1.263418,0.023524,0.023524,0.050655
2020-08-27 10:15:00+05:30,23592.599609,23625.050781,23581.300781,23605.400391,23605.400391,0,23605.560156,NaN,NaN,NaN,NaN,NaN,3,4.074511,0.050655,0.050655,-0.078165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-19 14:15:00+05:30,29109.449219,29111.349609,28861.449219,28863.849609,28863.849609,0,29161.059766,29343.845866,0.993771,NaN,NaN,NaN,3,23.448736,-0.850523,-0.850523,0.825082
2020-11-19 14:30:00+05:30,28865.150391,29120.199219,28831.449219,29102.000000,29102.000000,0,29116.219922,29313.308431,0.993276,NaN,NaN,NaN,3,38.622103,0.825082,0.825082,-0.226614
2020-11-19 14:45:00+05:30,29102.199219,29150.849609,28984.550781,29036.050781,29036.050781,0,29062.600000,29278.575195,0.992623,NaN,NaN,NaN,3,36.466510,-0.226614,-0.226614,-0.416555


In [ ]:
df2=df.copy()
#df2.to_excel('develop_strat.xlsx')
df2.to_csv('develop_strat.csv')

In [ ]:
from google.colab import files
files.download('develop_strat.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### ***Use Tableau to derive parameters of the indicators for your strategy***
##### Upload dataframe file in Tableau
##### Do the reverse engineering to derive the parameters
##### Use derived parameters in the code and verify



**Return Calculations** 

In [ ]:
# Signal and position generation
df2['signal_long']=(df2['sma_ratio']>0.9999)&(df2['sma_ratio']<1.0604)&(df2['RSI_14']>52.9)&(df2['RSI_14']<85)&(df2['Close']>df2['Open'])&(df2['weekday']!=2)
df2['position']=df2['signal_long'].shift(1)

In [ ]:
# Calculating returns and equity for the buy and hold case
df2['bnh_returns']= df2['Close'].pct_change()
df2['bnh_equity'] = np.cumprod(1+df2['bnh_returns'])

In [ ]:
# Calculating returns and equity for the strategy
df2['strategy_returns'] = df2['bnh_returns'] * df2['position']
df2['strategy_equity'] = np.cumprod(1+df2['strategy_returns'])

In [ ]:
#df2['strategy_equity']
#df2.to_excel('develop_strat.xlsx')

#### ***Analysis of performance of the strategy***
##### Graphical representation of the strategy returns
##### Comparative Analysis of strategy returns vs buy/hold of the instrument



In [ ]:
#total bnh return in %
print('Total Buy and Hold return:',np.round((df2['bnh_equity'][-1] - 1)*100,2),'%')

IndexError: ignored

In [ ]:
#total strategy return in %
print('Total strategy return:',np.round((df2['strategy_equity'][-1] - 1)*100,2),'%')

In [ ]:
df2[['bnh_equity','strategy_equity']].plot(figsize=(15,10),grid=True)

In [ ]:
'''
# Validating using normal total change formula
((df2['Close'][-1] - df2['Close'][0])/df2['Close'][0])*100
'''